# Data Handling
You'll find here:

-Data loading

-Data cleaning

-SQL Requetes

In [1]:
# BASIC IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nbresult import ChallengeResult

In [5]:
# DOWNLOAD FROM INTERNET

!curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/tv_movies_popularity.csv \
    --output tv_movies_popularity.csv
!mv tv_movies_popularity.csv raw_data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1978k  100 1978k    0     0  1800k      0  0:00:01  0:00:01 --:--:-- 1800k
